In [39]:
import os
import numpy as np
import pandas as pd
from ape import accounts, chain, Contract, networks, project

In [16]:
# SEE: https://gist.github.com/banteg/dcf6082ff7fc6ad51ce220146f29d9ff
networks.parse_network_choice('ethereum:mainnet:alchemy').__enter__()

<alchemy chain_id=1>

In [20]:
chain.blocks.head

Block(num_transactions=148, hash=HexBytes('0x3d92fc5475d17c48a61f43f396adc3feaf460cda6f50aa24d8cc30b98218cec9'), number=16636366, parent_hash=HexBytes('0x106a1670db845c741444d2793af525d49d6b41cd4e5d355a2e66f28f96091ad7'), size=63949, timestamp=1676491019, gas_limit=30000000, gas_used=11507736, base_fee=40873016887, difficulty=0, total_difficulty=58750003716598352816469)

In [23]:
wad = int(1e18)
wad

1000000000000000000

In [21]:
tricrypto = Contract("0xD51a44d3FaE010294C616388b506AcdA1bfAAE46")

In [24]:
print(f"Prices now: {[tricrypto.price_oracle(i) / wad for i in range(2)]}")

Prices now: [23562.7627212254, 1624.7742984373592]


In [42]:
# get prices for the last 3 months sampling every 10 min
FILENAME = f"data/prices.csv"

In [47]:
start_block = 15988366
is_head = True
if os.path.exists(FILENAME):
    is_head = False
    df = pd.read_csv(FILENAME)
    last_block = df['block_number'].iloc[len(df)-1]  # resume in case we error out below for some reason
    start_block = last_block + 50

In [48]:
blocks = range(start_block, 16636366, 50)  # 3 months ago to now with step of 50 blocks (~ 10 min)
for block in blocks:
    print(f"Processing block {block} ...", end='\r')

    # get the prices from tricrypto oracle at block
    prices = [tricrypto.price_oracle(i, block_identifier=block) / wad for i in range(2)]
    data = {'block_number': [block], 'price0': [prices[0]], 'price1': [prices[1]]}
    
    # convert to pd dataframe then append to filename
    df = pd.DataFrame(data=data)
    df.to_csv(FILENAME, mode='a', index=False, header=is_head)
    
    if is_head:
        is_head = False